In [1]:
import os
import torch
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torch.utils.data import DataLoader
from modules.lifter_2d_3d.dataset.drive_and_act_keypoint_dataset import DriveAndActKeypointDataset
from modules.lifter_2d_3d.model.linear_model.lit_linear_model import LitSimpleBaselineLinear
from modules.utils.visualization import (
    plot_samples
)
from IPython.display import display
from pathlib import Path


pl.seed_everything(1234)

# ------------
# dataset path
# ------------
dataset_root_path = Path('/root/data/processed/drive_and_act/')
keypoint_2d_path = dataset_root_path / 'keypoint_detection_results'
keypoint_3d_path = dataset_root_path / 'annotations'
# ------------
# model
# ------------
image_width = 1280
image_height = 1024
batch_size = 64
max_epoch = 200
val_check_period = 5
early_stopping_patience = 5
# ------------
# saved model path
# ------------
pretrained_model_path = './saved_lifter_2d_3d_model/synthetic_cabin_ir/A_Pillar_Codriver/prediction/linear_model/'
with open(f'{pretrained_model_path}/best_model_path.txt', 'r') as f:
    checkpoint_path = f.readline()


Global seed set to 1234


In [2]:
def run_experiment(
    train_actors,
    val_actors,
    test_actors,
    is_plot_gt_skeleton=False,
    model_suffix=''
):
    train_dataset = DriveAndActKeypointDataset(
        prediction_file=(keypoint_2d_path / 'keypoint_detection_train.json').as_posix(),
        annotation_file=(keypoint_3d_path / 'person_keypoints_train.json').as_posix(),
        image_width=image_width,
        image_height=image_height,
        actors=train_actors,
        exclude_ankle=True,
        exclude_hip=True
    )
    val_dataset = DriveAndActKeypointDataset(
        prediction_file=(keypoint_2d_path / 'keypoint_detection_train.json').as_posix(),
        annotation_file=(keypoint_3d_path / 'person_keypoints_train.json').as_posix(),
        image_width=image_width,
        image_height=image_height,
        actors=val_actors,
        exclude_ankle=True,
        exclude_hip=True
    )
    test_dataset = DriveAndActKeypointDataset(
        prediction_file=(keypoint_2d_path / 'keypoint_detection_train.json').as_posix(),
        annotation_file=(keypoint_3d_path / 'person_keypoints_train.json').as_posix(),
        image_width=image_width,
        image_height=image_height,
        actors=test_actors,
        exclude_ankle=True,
        exclude_hip=True
    )

    print(
        'train_dataset', len(train_dataset),
        'val_dataset', len(val_dataset),
        'test_dataset', len(test_dataset)
    )
    train_loader = DataLoader(train_dataset, batch_size=batch_size, drop_last=True, shuffle=True, num_workers=24)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, drop_last=True, num_workers=24)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=24)

    all_activities = train_dataset.activities.union(val_dataset.activities).union(test_dataset.activities)
    lit_model = LitSimpleBaselineLinear(exclude_ankle=True, exclude_hip=True, all_activities=all_activities)
    lit_model.load_from_checkpoint(checkpoint_path)

    model_checkpoint = ModelCheckpoint(monitor='val_loss',mode='min', save_top_k=1)
    early_stopping = EarlyStopping(monitor="val_loss", mode="min", patience=early_stopping_patience)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    saved_model_path = f'./saved_lifter_2d_3d_model/drive_and_act/transfer_learning/synthetic_cabin_ir/full/linear_model_{model_suffix}'
    if not os.path.exists(saved_model_path):
        os.makedirs(saved_model_path)
    trainer = pl.Trainer(
        # max_steps=10,
        max_epochs=max_epoch,
        callbacks=[model_checkpoint, early_stopping],
        accelerator=device,
        check_val_every_n_epoch=val_check_period,
        default_root_dir=saved_model_path,
        gradient_clip_val=1.0
    )
    trainer.fit(lit_model, train_loader, val_loader)
    with open(f'{saved_model_path}/best_model_path.txt', 'w') as f:
        f.writelines(model_checkpoint.best_model_path)
    best_checkpoint_path = model_checkpoint.best_model_path
    trainer.test(ckpt_path=best_checkpoint_path, dataloaders=test_loader)
    return trainer

## Fine-Tuned with full dataset

In [3]:
trainer = run_experiment(
    train_actors=['vp1', 'vp2', 'vp3', 'vp4', 'vp5', 'vp6', 'vp7', 'vp8'],
    val_actors=['vp9', 'vp10', 'vp15'],
    test_actors=['vp11', 'vp12', 'vp13', 'vp14'],
    is_plot_gt_skeleton=False,
    model_suffix='all_actor'
)

skipping problematic image 15783
skipping problematic image 17258
skipping problematic image 17259
skipping problematic image 21271
skipping problematic image 21272
skipping problematic image 21273
skipping problematic image 21274
skipping problematic image 21275
skipping problematic image 21276
skipping problematic image 33527
train_dataset 22885 val_dataset 6240 test_dataset 11018


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: saved_lifter_2d_3d_model/drive_and_act/transfer_learning/synthetic_cabin_ir/full/linear_model_all_actor/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type          | Params
----------------------------------------
0 | model | BaselineModel | 4.3 M 
----------------------------------------
4.3 M     Trainable params
0         Non-trainable params
4.3 M     Total params
17.105    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

check #0
val MPJPE from: 128 samples : 2767.831563949585


Training: 0it [00:00, ?it/s]

...

Restoring states from the checkpoint path at saved_lifter_2d_3d_model/drive_and_act/transfer_learning/synthetic_cabin_ir/full/linear_model_all_actor/lightning_logs/version_0/checkpoints/epoch=49-step=17850.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at saved_lifter_2d_3d_model/drive_and_act/transfer_learning/synthetic_cabin_ir/full/linear_model_all_actor/lightning_logs/version_0/checkpoints/epoch=49-step=17850.ckpt


Testing: 0it [00:00, ?it/s]

/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/conda/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


MPJPE: 59.17740461015161
PJPE
                     PJPE
nose            51.009483
left_eye        51.110857
right_eye       47.491245
left_ear        14.587826
right_ear       42.581240
left_shoulder   41.818321
right_shoulder  39.374703
left_elbow      66.735620
right_elbow     65.013455
left_wrist      80.595379
right_wrist     98.196295
left_hip        38.181280
right_hip       37.041859
activities_mpjpe:
{'closing_door_inside': 88.13386517388169, 'entering_car': 120.06722474472981, 'sitting_still': 47.649342576343585, 'closing_bottle': 54.98819176352304, 'drinking': 52.54070091849804, 'pressing_automation_button': 58.444250967588715, 'fetching_an_object': 103.31589860495374, 'opening_bottle': 60.81968477970338, 'eating': 49.55149762248152, 'preparing_food': 69.32677151940382, 'opening_backpack': 109.02917219074216, 'placing_an_object': 93.35297463217618, 'reading_newspaper': 68.31565831123788, 'taking_off_sunglasses': 72.59146962460392, 'writing': 63.44236587283765, 'using_multimed

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│           mpjpe           │     59.17740461015161     │
└───────────────────────────┴───────────────────────────┘

In [4]:
trainer.model.test_history[0]['mpjpe']


59.17740461015161

In [5]:
trainer.model.test_history[0]['pjpe']

,PJPE
nose,51.009483
left_eye,51.110857
right_eye,47.491245
left_ear,14.587826
right_ear,42.581240
left_shoulder,41.818321
right_shoulder,39.374703
left_elbow,66.735620
right_elbow,65.013455
left_wrist,80.595379


In [11]:
pd.DataFrame(trainer.model.test_history[0]['activities_mpjpe'], index=['mpjpe']).T.sort_index()

,mpjpe
closing_bottle,54.988192
closing_door_inside,88.133865
closing_laptop,70.303653
drinking,52.540701
eating,49.551498
entering_car,120.067225
exiting_car,189.303968
fastening_seat_belt,83.317250
fetching_an_object,103.315899
interacting_with_phone,52.715296
